<a href="https://colab.research.google.com/github/sanjaynagi/AnoFold/blob/main/notebooks/AnoFold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip install py3Dmol

# !wget https://github.com/gnina/gnina/releases/download/v1.1/gnina
# !chmod +x gnina
# !./gnina --version

In [ ]:
# # genes = ['AGAP006227', 'AGAP006228', 'AGAP006723', 'AGAP006724', 'AGAP006725', 'AGAP006726', 'AGAP006727']
# df_genes = pd.read_csv("https://raw.githubusercontent.com/sanjaynagi/AnoExpress/main/resources/AgamP4.annots.tsv", 
#                        sep="\t")

# search = 'S-transferase'
# genes = df_genes.fillna('bl').query('GeneDescription.str.contains(@search)')['GeneID'].to_list()

In [ ]:
# ligands = ['ddt3d', 'deltamethrin3d']           #, 'pirimiphos-methyl-oxon3d', 'malathion3d', 'cis-permethrin3d','transfluthrin3d']
# genes = ['AGAP006227']
# # for gene in genes:
# #     for ligand in ligands:
# #           res = invin(gene, ligand=ligand, override_desc='glutathione transferase')

## iMOVE - inverse molecular docking with alphafold and gnina

In [ ]:
import imove

for ligand in ['ddt3d', 'deltamethrin3d']:
    for mutagen_dict in [{114:'T'}, {120:'L'}]:
        imove.dock("AGAP009194", ligand, wkdir="../../", verbose=True, mutagenesis_dict=mutagen_dict)

### Chimera / HEM / Superimposement

In [ ]:
# %pip install chimerax

In [ ]:
# from chimerax.core.commands import run

# # Open your PDB and the reference PDB with the heme
# run(session, pdb_path)
# run(session, ref_p450_path)

# # Assuming your P450 is model #1 and the reference is model #2
# # Perform the superposition
# run(session, "match #2 to #1")

# # Select the heme from the reference model
# run(session, "select #2/name HEM")

# # Copy the selected heme to your P450 model
# run(session, "copy sel models #1")

# # Optionally, you can hide or close the reference model
# run(session, "hide #2")
# # or
# # run(session, "close #2")

# # Save the result
# run(session, "save your_p450_with_heme.pdb models #1")